In [1]:
import pandas as pd
import requests

In [2]:
df_lst = []
columns = ['aid', 'tid', 'tname', 'title', 'duration', 'mid', 'name', 'view', 'danmaku', 'reply', 'favorite', 'coin', 'share', 'like', 'short_link_v2', 'pub_location', 'pubdate', 'rcmd_reason']

In [ ]:
def fetch_data(
    page: int
):
    tar = f"https://api.bilibili.com/x/web-interface/popular/series/one?number={page}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
        # 'Cookie': 
    }
    response = requests.get(tar, headers=headers)
    data = response.json()
    if data.get('code') == 0 and 'data' in data and 'list' in data.get('data', {}):
        data_list = data['data']['list']
        for item in data_list:
            tmp = []
            tmp.append(item.get('aid', None))                      # 视频ID
            tmp.append(item.get('tid', None))                      # 分区ID
            tmp.append(item.get('tname', None))                    # 分区
            tmp.append(item.get('title', None))                    # 标题     
            tmp.append(item.get('duration', None))                 # 时长 s
            
            # 处理嵌套字典
            owner = item.get('owner', {})
            tmp.append(owner.get('mid', None))                    # up主id
            tmp.append(owner.get('name', None))                   # up主名称

            # 处理统计数据
            stat = item.get('stat', {})
            tmp.append(stat.get('view', None))                    # 播放量
            tmp.append(stat.get('danmaku', None))                 # 弹幕量
            tmp.append(stat.get('reply', None))                   # 评论量
            tmp.append(stat.get('favorite', None))                # 收藏量
            tmp.append(stat.get('coin', None))                    # 硬币量
            tmp.append(stat.get('share', None))                   # 分享量
            tmp.append(stat.get('like', None))                    # 点赞量

            tmp.append(item.get('short_link_v2', None))           # 短链接
            tmp.append(item.get('pub_location', None))            # 位置
            tmp.append(item.get('pubdate', None))                 # 发布时间
            tmp.append(item.get('rcmd_reason', None))             # 推荐理由
            df_lst.append(tmp)
    else:
        print(f"Error fetching data of page {page}: {data.get('message', 'Unknown error')}")

In [6]:
for i in range(1, 322):
    try:
        fetch_data(i)
    except Exception as e:
        print(f"Error fetching data of page {i}: {e}")
        continue

Error fetching data of page 105: 啥都木有


In [7]:
df = pd.DataFrame(df_lst, columns=columns)
df.index = df.index + 1
df.to_csv('bilibili.csv', index=True, encoding='utf-8-sig')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11564 entries, 1 to 11564
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   aid            11564 non-null  int64 
 1   tid            11564 non-null  int64 
 2   tname          11564 non-null  object
 3   title          11564 non-null  object
 4   duration       11564 non-null  int64 
 5   mid            11564 non-null  int64 
 6   name           11564 non-null  object
 7   view           11564 non-null  int64 
 8   danmaku        11564 non-null  int64 
 9   reply          11564 non-null  int64 
 10  favorite       11564 non-null  int64 
 11  coin           11564 non-null  int64 
 12  share          11564 non-null  int64 
 13  like           11564 non-null  int64 
 14  short_link_v2  11564 non-null  object
 15  pub_location   7550 non-null   object
 16  pubdate        11564 non-null  int64 
 17  rcmd_reason    11564 non-null  object
dtypes: int64(12), object(6)
me